# Sequence generation for the pseudoRWM set of experiments

## Import statements and utilities

In [ ]:
import numpy as np
import matlab.engine

In [ ]:
use_matlab = True  # use matlab to call createstimsequence?
if use_matlab:
    m = matlab.engine.start_matlab()
    m.addpath(m.genpath("C:/Gaia/CCN Lab/Utilities/sequences/pseudoRWM/"), nargout=0)
else:
    from numpy.matlib import repmat

def shuffle_along_axis(arr, axis):
    idx = np.random.rand(*arr.shape).argsort(axis=axis)
    return np.take_along_axis(arr,idx,axis=axis)

def shuffled(arr):
    arr_shuffled = arr.copy()
    np.random.shuffle(arr_shuffled)
    return(arr_shuffled)

## Settings

In [ ]:
exp_type = "pseudoRWM"  # choose from ["pseudoRWM", "pseudoRWMCtrl", "pseudoRWMConf", "pseudoRWMReps"]
num_conditions = 10 # number of different file sequences to generate (change if needed)

reps_dict = {"pseudoRWM": 12, "pseudoRWMCtrl": 12, "pseudoRWMConf": 11, "pseudoRWMReps": 11}  # number of repetitions after the first presentation
exact_reps =  True if exp_type in ["pseudoRWMConf", "pseudoRWMReps"] else False  # should the number of repetitions be exact or ok to exceed by 1?
reps = reps_dict[exp_type] # number of stimulus repetitions (after first presentation; change if needed)

## Main sequence creation function

In [ ]:
class pseudoRWMSequenceMaker:

    def __init__(self, exp_type, num_reps, use_matlab=True, num_conditions=10, exact_reps=False):

        # ================ SETTINGS ==========================================================================
        assert exp_type in ["pseudoRWM", "pseudoRWMCtrl", "pseduoRWMConf", "pseudoRWMReps"], f"{exp_type} is not a valid exp_type"
        self.exp_type = exp_type
        self.num_reps = num_reps
        self.num_conditions = num_conditions
        self.use_matlab = use_matlab
        self.exact_reps = exact_reps
        self.to_dir = f"C:/Gaia/CCN Lab/Utilities/sequences/pseudoRWM/{exp_type}/"

        self.num_keys = 3
        self.max_stims = 6
        self.block_structure = np.array([6, 6, 6, 6, 6, 6]) # 6 blocks: 3*type (Points vs Goals)
        self.num_blocks = len(self.block_structure)
        self.trial_type_structure = np.arange(self.num_blocks)%2
        self.num_goals_blocks = np.sum(self.trial_type_structure)
        self.num_goals_trials = np.sum(self.block_structure[np.where(self.trial_type_structure)])*(self.num_reps+1)

        # R: columns represent keyboard keys
        # each column says how many stimuli will be associated with that key
        # e.g., 1, 2, 3 means 1 stimulus will be associated with key 0, 2, with key 1, 3 with key 2, 
        # for a total of 6 items
        if self.exp_type in ["pseudoRWM", "pseudoRWMCtrl"]:
            self.R = np.vstack((
                np.array([1, 2, 3]),
                np.array([2, 2, 2]), 
                np.array([1, 2, 3]),
                np.array([2, 2, 2]), 
                np.array([1, 2, 3]),
                np.array([2, 2, 2])
            ))
        else:
            self.R = np.vstack((
                np.array([2, 2, 2]), 
                np.array([2, 2, 2]), 
                np.array([2, 2, 2]), 
                np.array([2, 2, 2]), 
                np.array([2, 2, 2]), 
                np.array([2, 2, 2])
            ))

        # goal images  
        self.all_fracts = np.arange(1, (self.num_goals_trials*2)+1)  # all fractal image numbers 
        self.all_ctrl_goal_sets = [[5, 17], [25, 26], [24, 30], [56, 59], [76, 77]]*2 # fractals to use in the control version includes goal and non-goal

    def make_sequences(self):
        for s_i in range(self.num_conditions):
            # ================ BLOCKS ===========================================================================
            # blocks: array of length n with each element representing the block's set size
            blocks = self.block_structure.copy()

            # ================ TRIAL TYPES =======================================================================
            # trial_types: array  where each element represent the sequence
            # of trial types (1 = Points, 0 = Goals) a participant will experience
            trial_types = self.trial_type_structure.copy()
            for i in range(0, self.num_blocks//2, 2):
                np.random.shuffle(trial_types[i:i+2]) 

            # ================ RULES =============================================================================
            # block_rules: a list where each element is a dictionary, with keys representing a stimulus and
            # variables representing the solution for each stimulus
            # here we shuffle within rows (only the first three columns), so that it's not always the same keys 
            # that have 1, 2, or 3 stimuli associated with them (no difference for rows with 2, 2, 2) 
            R_i = shuffle_along_axis(self.R, axis=1)  # mix up stim/action within the rule
            block_rules = []
            for block_i in range(self.num_blocks):
                block_rules.append({i: k for i, k in enumerate([key_i for key_i in range(self.num_keys) for _ in range(R_i[block_i, key_i])])})

            # ================ STIMULI ===========================================================================
            # stim_sets: stimulus sets (folders from where images will be taken for each block)
            stim_sets = (np.random.permutation(self.num_blocks)+1)
        
            # block_stimuli: will contain dictionaries with an image number for each stimulus
            block_stimuli = []
            for block_i, ns in enumerate(blocks):
                block_stimuli.append({i: s for i, s in enumerate((np.random.permutation(self.max_stims)+1)[0:ns])})

            # block_seqprototypes: will contain dictionaries for each participant, with keys representing a set size and
            # variables as lists with a sequence of stimuli to be presented
            # create a prototype (corresponding to stimuli rather than stimulus images) for each set size
            block_seqprototypes = []
            
            # block_sequences: maps block_seqprototypes to corresponding stimulus sequences based on block_stimuli
            block_sequences = []
            
            for block_i, ns in enumerate(blocks):
                criterion_passed = False
                if self.use_matlab:
                    while not criterion_passed:
                        temp_seqprototype = np.squeeze(np.array(m.createstimsequence(m.double(int(self.num_reps)), m.double(int(ns))))).astype(int)
                        criterion_passed = np.all(np.unique(temp_seqprototype, return_counts=True)[1]==(self.num_reps+1)) or (not self.exact_reps)
                    block_seqprototypes.append(temp_seqprototype)
                else:
                    # worse (but faster) alternative if createstimsequence doesn't work
                    temp_seqprototype = []
                    for _ in range(self.num_reps+1):
                        temp_seqprototype = np.hstack((temp_seqprototype, (shuffled(np.arange(1, ns+1)))))
                    block_seqprototypes.append(temp_seqprototype)
                # turn into stimuli
                block_sequences.append(np.vectorize((block_stimuli[block_i]).get)(block_seqprototypes[block_i]-1))

            # ================ GOAL IMAGES ======================================================================
            all_images = shuffled(self.all_fracts)
            if self.exp_type == "pseudoRWM":
                goal_images = all_images[0:len(all_images)//2] 
                nongoal_images = all_images[len(all_images)//2:]

            elif self.exp_type == "pseudoRWMCtrl":
                goal_set = self.all_ctrl_goal_sets[s_i]
                # counterbalance which image is the goal and which one is the non-goal across participants
                goal_images = np.repeat(goal_set[int(s_i+1 >= self.num_conditions//2)], self.num_goals_trials)
                nongoal_images = np.repeat(goal_set[1-int(s_i+1 >= self.num_conditions//2)], self.num_goals_trials)

            elif self.exp_type == "pseudoRWMConf":
                pass

            elif self.exp_type == "pseudoRWMReps":
                pass

            # ================ CSV FILE ==========================================================================
            # create csv
            # rows: stim, correct key, set size, blocks, img_folders, img_nums, trial_type, goal_img, nongoal_img
            goal_stim_count = 0

            for block_i, ns in enumerate(blocks):
                block_length = (self.num_reps + 1) * ns  # number of trials in a block
                
                this_block = np.full((9, block_length), np.nan)

                _, unique_idx = np.unique(block_sequences[block_i], return_index=True)
                block_cond = trial_types[block_i]
                
                this_block[0] = block_seqprototypes[block_i]  # stimulus number
                this_block[1] = np.vectorize((block_rules[block_i]).get)(block_seqprototypes[block_i]-1) # correct key
                this_block[2] = np.repeat(ns, block_length)  # set size
                this_block[3] = np.repeat(block_i+1, block_length)  # block number
                this_block[4] = np.repeat(stim_sets[block_i], block_length)  # image folder
                this_block[5] = block_sequences[block_i]  # stimulus number
                this_block[6] = np.repeat(block_cond, block_length)  # trial type

                if block_cond == 0:
                    if self.exp_type in ["pseudoRWM", "pseudoRWMCtrl"]:
                        this_block[7] = goal_images[goal_stim_count:goal_stim_count+block_length]
                        this_block[8] = nongoal_images[goal_stim_count:goal_stim_count+block_length]
                    else:
                        pass
                    goal_stim_count += block_length

                if block_i == 0:
                    train_seq = this_block
                    unique_stims = this_block[:, unique_idx]
                else:
                    train_seq = np.column_stack((train_seq, this_block))
                    unique_stims = np.column_stack((unique_stims, this_block[:, unique_idx]))
        
        # np.savetxt(f"{self.to_dir}seq{s_i+1}_learning.csv", train_seq, delimiter=",")
        print(train_seq.shape)

seqmkr = pseudoRWMSequenceMaker("pseudoRWM", num_reps=11, num_conditions=2, use_matlab=True, exact_reps=True)
seqmkr.make_sequences()

## Quit MATLAB if needed

In [ ]:
# quit matlab
if use_matlab:
    m.exit()